In [27]:
import numpy as np

# TODO: Write shapes everywhere needed 
 
layer_sizes = [2, 3, 1]
num_layers = len(layer_sizes)


mu, sigma = 0, 0.1  # mean and standard deviation
weights = [] 
biases = []

# Generate start matrix for weights and biases 
for i in range(num_layers - 1):
    n_in = layer_sizes[i]           # shape 2x3
    n_out = layer_sizes[i + 1]      # shape 3x1
    weights.append(np.random.normal(mu, sigma, size=(n_out, n_in)))
    biases.append(np.random.normal(mu, sigma, size=(n_out, 1)))

# Activation function 
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 𝞂'(z) 
def sigmoid_prime(z): 
    return sigmoid(z) * (1 - sigmoid(z))

# Cost function 
def bce_loss(y, a, eps=1e-12):
    # y, a: (1, m)
    a = np.clip(a, a_min=eps, a_max=1-eps)  # Ensures that a never becomes 0
    m = y.shape[1]  # Number of inputs 
    return -1/m * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))

# XOR input data
x = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
y = np.array([[0, 1, 1, 0]])

def feedforward(X, weights, biases):
    A = X           # A^(0)
    A_list = [X]    # stores activations
    Z_list = []     # stores pre-activations
    
    for l in range(len(weights)): 
        Z = weights[l] @ A + biases[l]
        A = sigmoid(Z)
        Z_list.append(Z)
        A_list.append(A)
    return A_list, Z_list

def update_params(weights, biases, nabla_w, nabla_b, lr):
    for l in range(len(weights)):
        weights[l] -= lr * nabla_w[l]
        biases[l]  -= lr * nabla_b[l]
    return weights, biases


lr = 1.0
epochs = 100
for epch in range(epochs):     
    A_list, Z_list = feedforward(X=x, weights=weights, biases=biases)
    m = x.shape[1]
    loss = bce_loss(y, A_list[-1])
    # BP1: Compute delta for output layer 
    delta_L = A_list[-1] - y

    # Store deltas backwards 
    deltas = [delta_L]

    # BP2: Backpropagate trough hidden layers 
    for l in range(len(weights) - 1, 0, -1): 
        delta_next = deltas[0]  # Most recently computed delta (from layer l+1)
        z = Z_list[l - 1]       # Pre-activation at layer l
        delta = (weights[l].T @ delta_next) * sigmoid_prime(z)
        deltas.insert(0, delta) # Insert at the beginning

    # Store gradients
    nable_weights, nabla_biases = [], []

    for l in range(len(weights)):
        nable_weights.append((deltas[l] @ A_list[l].T) / m)                 # matches weights[l]
        nabla_biases.append(np.sum(deltas[l], axis=1, keepdims=True) / m)  # matches biases[l]
        
    weights, biases = update_params(weights, biases, nable_weights, nabla_biases, lr)
        
    if epch % 100 == 0: 
        print(epch, loss)
    
# Evaluate 
A_list, _ = feedforward(X=x, weights=weights, biases=biases)
probs = A_list[-1]
preds = (probs > 0.5).astype(int)

print("probs:", probs)
print("preds:", preds)
print("true :", y)

# y_hat = A_list[-1]
# print(f"Output shpae y_hat: {y_hat.shape}")
# print(f"Output of y_hat: {y_hat}")

0 0.6931442641906013
probs: [[0.49998158 0.50077793 0.49920365 0.49999111]]
preds: [[0 1 0 0]]
true : [[0 1 1 0]]


In [ ]:
class BinaryClassifer: 
    def __init__(self, layer_sizes, mu=0.0, sigma=0.01): 
        self.layer_sizes = layer_sizes 
        self.mu = mu
        self.sigma = sigma 
        self.weights = []
        self.biases  = []
        
        # Generate start matrix for weights and biases 
        for i in range(num_layers - 1):
            n_in = self.layer_sizes[i]           
            n_out = self.layer_sizes[i + 1]      
            self.weights.append(np.random.normal(mu, sigma, size=(n_out, n_in)))
            self.biases.append(np.random.normal(mu, sigma, size=(n_out, 1)))

        self.A_list = None 
        self.Z_list = None 
        self.loss_history = []
    
    # -- Activation function --- 
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_prime(self, z): 
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    # --- loss function ---
    def bce_loss(self, y, a, eps=1e-12):
        # y, a: (1, m)
        a = np.clip(a, a_min=eps, a_max=1-eps)  # Ensures that a never becomes 0
        m = y.shape[1]  # Number of inputs 
        return -1/m * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))

    # --- Feedforward--- 
    def feedforward(self, X):
        A = X           # A^(0)
        A_list = [X]    # stores activations
        Z_list = []     # stores pre-activations
        
        for l in range(len(weights)): 
            Z = self.weights[l] @ A + self.biases[l]
            A = sigmoid(Z)
            Z_list.append(Z)
            A_list.append(A)
            
            self.A_list, self.Z_list =A_list, Z_list
        return A
    
    # --- Backward propagation --- 
    def backward(self, y): 
        A_list, Z_list = feedforward(X=x, weights=weights, biases=biases)
        m = x.shape[1]

        # BP1: Compute delta for output layer 
        delta_L = A_list[-1] - y

        # Store deltas backwards 
        deltas = [delta_L]

        # BP2: Backpropagate trough hidden layers 
        for l in range(len(weights) - 1, 0, -1): 
            delta_next = deltas[0]  # Most recently computed delta (from layer l+1)
            z = Z_list[l - 1]       # Pre-activation at layer l
            delta = (weights[l].T @ delta_next) * sigmoid_prime(z)
            deltas.insert(0, delta) # Insert at the beginning

        # Store gradients
        nable_weights, nabla_biases = [], []

        for l in range(len(weights)):
            nable_weights.append((deltas[l] @ A_list[l].T) / m)                 # matches weights[l]
            nabla_biases.append(np.sum(deltas[l], axis=1, keepdims=True) / m)  # matches biases[l]
        return nable_weights, nabla_biases
    
    # --- update --- 
    def step(self, nabla_w, nabla_b, lr):
        for l in range(len(self.weights)):
            self.weights[l] -= lr * nabla_w[l]
            self.biases[l]  -= lr * nabla_b[l]
    
    # --- training --- 
    def fit(self, X, y, lr=1.0, epochs=1000, verbose=100): 
        for epoch in range(epochs): 
            yhat = self.feedforward(X)
            loss = self.bce_loss(y, yhat)
            self.loss_history.append(loss)
            
            nabla_w, nabla_b = self.backward(y)
            self.step(nabla_w, nabla_b, lr)
            if verbose is not None and epoch % verbose == 0:
                print(epoch, loss)
        return self
    
    def predict_proba(self, X): 
        return self.feedforward(X)
    
    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) > threshold).astype(int)


        